<img src='../../../../Other/img/dropout.png'>

&emsp;&emsp;可以看出,在上图中,通过随机减少每一层神经元的数目,能够把稠密的神经网络(左)变成稀疏的连接(右).
神经网络的复杂性和神经元的个数有关,神经元个数越多,模型越复杂,也越容易过拟合.
如果能够减少神经元的数目,那么就能减少神经网络过拟合的倾向.

&emsp;&emsp;减少神经网络的连接实现起来相对比较复杂.一个等价的最简单的方式是把Dropout层输入张量的某些的元素随机置为零(使用一个和原始张量一样大小的掩码张量来实现).在训练过程中,这样做的结果可以使模型不容易过拟合.
为了校正张量的元素被置为零造成的影响,需要对张量所有的元素做缩放.假设元素被随机置零的概率为$p$(一般设置为0.5),
那么这个缩放因子应该是$1/(1-p)$.也就是说,丢弃层首先会根据输入张量的形状,以及元素置零的概率产生一个随机的掩码张量,然后把输入张量和这个掩码张量按元素相乘,
最后对结果张量乘以缩放因子,输出最终结果,即为丢弃层的过程.
在预测过程(forward)中,由于丢弃层会降低神经网络的准确率,所以应该跳过丢弃层(或者把随机置零的概率设置为零).
在PyTorch中,可以通过调用nn.Module的trian方法和eval方法来切换丢弃层的训练和预测状态.

<font color='red' size=4>集成学习角度的解释</font>:

&emsp;&emsp;每做一次丢弃(若$p=0.5$),相当于从原始的网络中采样得到一个子网络.如果一个神经网络有$n$个神经元,那么总共可以采样出$2^n$个子网络.
每次迭代都相当于训练一个不同的子网络,这些子网络都共享原始网络的参数.那么,最终的网络可以近似看作集成了指数级个不同网络的组合模型．

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

a = torch.ones((10, 10))
drop_5 = F.dropout(a,
                   p=0.5)  # 元素归零的概率
drop_5  # 进行了放缩,1 * (1/(1-0.5)) = 2

tensor([[0., 2., 0., 0., 2., 2., 0., 0., 2., 2.],
        [2., 2., 0., 0., 2., 2., 0., 0., 0., 2.],
        [0., 2., 0., 2., 2., 0., 2., 2., 0., 2.],
        [2., 2., 2., 0., 2., 0., 2., 0., 0., 2.],
        [0., 2., 0., 2., 2., 2., 2., 2., 0., 2.],
        [0., 0., 0., 0., 0., 2., 2., 2., 2., 0.],
        [0., 2., 2., 0., 2., 0., 2., 2., 2., 0.],
        [0., 0., 2., 2., 0., 0., 2., 2., 2., 2.],
        [0., 0., 0., 0., 0., 2., 0., 2., 2., 2.],
        [2., 0., 0., 2., 0., 0., 2., 2., 0., 2.]])

In [2]:
drop_1 = F.dropout(a, p=0.9)
drop_1  # 进行了放缩,1 * (1/(1-0.9)) = 10

tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 10.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 10.],
        [ 0.,  0.,  0.,  0.,  0., 10.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [10.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0., 10.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [3]:
drop_F = F.dropout(a, p=0.9,
                   training=False)  # apply dropout if is True. Default: True
drop_F

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [4]:
'''
# 实现方式:
def forward(self, input):
    return F.dropout(input, self.p,
                     self.training, # nn.Mudule的training属性
                     self.inplace)
'''
# During training, randomly zeroes some of the elements of the input tensor with probability p using samples from a Bernoulli distribution.
# Each channel will be zeroed out independently on every forward call.
Drop = nn.Dropout(p=0.5)
Drop(a)

tensor([[2., 2., 0., 2., 0., 2., 0., 2., 0., 0.],
        [0., 2., 2., 2., 2., 0., 0., 0., 0., 0.],
        [2., 2., 2., 2., 0., 2., 2., 0., 2., 0.],
        [2., 0., 2., 2., 0., 2., 0., 2., 2., 2.],
        [2., 2., 2., 0., 2., 2., 2., 0., 2., 2.],
        [2., 2., 2., 2., 2., 0., 2., 0., 0., 0.],
        [2., 0., 2., 2., 2., 2., 0., 2., 0., 0.],
        [2., 2., 0., 2., 2., 0., 0., 0., 2., 0.],
        [2., 0., 0., 2., 0., 2., 2., 0., 0., 0.],
        [2., 2., 0., 2., 0., 0., 0., 0., 2., 0.]])